這次的作業是這樣的，希望你能利用本週教的keras functional API，

設計一個mnist的分類模型，這個模型可以一次訓練十個獨立的全連接層，

每一個獨立的全連接層的目標是學習答案的十維向量裡的其中一個element，

每個全連接層有十個神經元，且都跟隨一個average layer把它的輸出做平均，

最後再把這十個層的輸出連接起來，取softmax之後比較結果。

In [10]:
#keras 
import keras
import keras.backend as K
from keras.models import Model
from keras.layers import Dense, Activation, Flatten, Permute, Conv1D, Conv2D, Add, Conv2DTranspose 
from keras.optimizers import Adam
from keras.layers import Input, Dense, concatenate, Lambda, Conv2D, Reshape, BatchNormalization, Lambda, Activation 
from keras.callbacks import ModelCheckpoint
from keras.regularizers import l2
from keras.activations import softmax
from keras.datasets import mnist
from keras.utils import plot_model

# tensorflow
import tensorflow as tf

#numpy
import numpy as np

In [11]:
# 把mnist load 進來
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

# 把每個像素的值從(0~255)->(0, 1)
x_train /= 255
x_test /= 255

print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

x_train shape: (60000, 28, 28)
60000 train samples
10000 test samples


In [12]:
# 定義一下Average
# 喔，keepdims這個參數在這裏真是他X的重要
def average_function(inputs):
    return K.mean(inputs, axis=-1, keepdims=True)

#把定義好的Average用Lambda封裝起來變成layer
average_layer = Lambda(average_function)

In [13]:
# Model 
model_input = Input(shape=(28, 28))
flatten = Flatten()(model_input)

# 作業開始

In [14]:
# 請給我十個分開的Dense層，每個層裡面有十個神經元，且輸入都來自flatten
# activation function請用relu
Dense_layer_list = []

for i in range(0, 10) :   #list from 0 to the upper-bound minu 1 -> rang (i, j) i ~ j-1
    layer_dense_new = Dense(10, activation = 'relu')
    x = layer_dense_new(flatten)
    Dense_layer_list.append(x)

# 每一個Dense層後面都必須再接一個 average_layer，然後把這十個層整理成一個list
# 舉例： 
# x_1 = Dense(10)
# avg_1 = averge_layer(x_1)
# list.append(...)
# 這不是完整的答案，只是個提示。
# 且答案不侷限於上述寫法，有更簡潔明快的做法
average_list = []

for i in range (0, 10) :
    avg_new = average_layer(Dense_layer_list[i])
    average_list.append(avg_new)

# 請把這十個層用學長教過的concatenate層連接起來（基本上就是把上面的average_list放進去並加上其他參數。）
concatenate_layer = concatenate(average_list)

# Hint: 這次的作業就只有上面的 pass 跟 None 的部分，
# 當然你也可以大方地使用 python 的 list comprehension 功能，
# 這樣可以少寫幾行。


# 作業結束

In [15]:
#加一個softmax
model_output = Activation("softmax")(concatenate_layer)

model = Model(model_input, model_output)


# 你的model.summary理論上會長這樣
print(model.summary())

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 28, 28)       0                                            
__________________________________________________________________________________________________
flatten_2 (Flatten)             (None, 784)          0           input_2[0][0]                    
__________________________________________________________________________________________________
dense_11 (Dense)                (None, 10)           7850        flatten_2[0][0]                  
__________________________________________________________________________________________________
dense_12 (Dense)                (None, 10)           7850        flatten_2[0][0]                  
____________________________________________________________________________________________

In [16]:
# compile & fit
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer="adam",
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=32,
          epochs=10,
          verbose=1,
          validation_data=(x_test, y_test))


Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 7s 116us/step - loss: 0.4990 - accuracy: 0.8798 - val_loss: 0.2973 - val_accuracy: 0.9190
Epoch 2/10
60000/60000 [==============================] - 7s 114us/step - loss: 0.2883 - accuracy: 0.9207 - val_loss: 0.2586 - val_accuracy: 0.9287
Epoch 3/10
60000/60000 [==============================] - 7s 111us/step - loss: 0.2565 - accuracy: 0.9288 - val_loss: 0.2410 - val_accuracy: 0.9311
Epoch 4/10
60000/60000 [==============================] - 7s 110us/step - loss: 0.2379 - accuracy: 0.9336 - val_loss: 0.2293 - val_accuracy: 0.9345
Epoch 5/10
60000/60000 [==============================] - 7s 110us/step - loss: 0.2238 - accuracy: 0.9374 - val_loss: 0.2202 - val_accuracy: 0.9367
Epoch 6/10
60000/60000 [==============================] - 9s 151us/step - loss: 0.2122 - accuracy: 0.9408 - val_loss: 0.2116 - val_accuracy: 0.9396
Epoch 7/10
60000/60000 [==============================] - 10s

In [17]:
average_list

[<tf.Tensor 'lambda_2/Mean:0' shape=(?, 1) dtype=float32>,
 <tf.Tensor 'lambda_2_1/Mean:0' shape=(?, 1) dtype=float32>,
 <tf.Tensor 'lambda_2_2/Mean:0' shape=(?, 1) dtype=float32>,
 <tf.Tensor 'lambda_2_3/Mean:0' shape=(?, 1) dtype=float32>,
 <tf.Tensor 'lambda_2_4/Mean:0' shape=(?, 1) dtype=float32>,
 <tf.Tensor 'lambda_2_5/Mean:0' shape=(?, 1) dtype=float32>,
 <tf.Tensor 'lambda_2_6/Mean:0' shape=(?, 1) dtype=float32>,
 <tf.Tensor 'lambda_2_7/Mean:0' shape=(?, 1) dtype=float32>,
 <tf.Tensor 'lambda_2_8/Mean:0' shape=(?, 1) dtype=float32>,
 <tf.Tensor 'lambda_2_9/Mean:0' shape=(?, 1) dtype=float32>]

In [18]:
concatenate_layer

<tf.Tensor 'concatenate_2/concat:0' shape=(?, 10) dtype=float32>